<a href="https://colab.research.google.com/github/Wayne-wyyking888/Stat-8931-GenAI/blob/main/chapter2/Chapter_II_Large_language_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Packages, Paths & Environment

### (1). Download the dependent `.py` files; trained model; tokenizer to the **currect directory**

In [1]:
# download model.py and tokenizer.py to the default directory
! gdown --id 1SU7jSZI36KGwBv5-zgc3WkStPK6lGKwL -O /content/model.py # download model.py
! gdown --id 1uXCgdmip79J6efM5hiHGCy9mdr_U8BXT -O /content/tokenizer.py # download tokenizer.py

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1SU7jSZI36KGwBv5-zgc3WkStPK6lGKwL
From (redirected): https://drive.google.com/uc?id=1SU7jSZI36KGwBv5-zgc3WkStPK6lGKwL&confirm=t&uuid=060aabd3-a1f3-46fc-b5b7-2fe3c6e1335b
To: /content/model.py
100% 13.3k/13.3k [00:00<00:00, 26.1MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1uXCgdmip79J6efM5hiHGCy9mdr_U8BXT
From (redirected): https://drive.google.com/uc?id=1uXCgdmip79J6efM5hiHGCy9mdr_U8BXT&confirm=t&uuid=cd7c0bae-e848-48db-9eb4-b8dc4389d705
To: /content/tokenizer.py
100

In [2]:
# trained language model (checkpoints) and tokenizer files under suitable directories (download and unzip files first !!)
import gdown
import zipfile
import os

def download_and_unzip(file_id, output_dir=None):

    if output_dir is None:
        output_dir = os.getcwd()

    os.makedirs(output_dir, exist_ok=True)

    # Download file
    url = f'https://drive.google.com/uc?id={file_id}'
    output = os.path.join(output_dir, 'temp.zip')
    gdown.download(url, output, quiet=False)

    # Unzip file
    with zipfile.ZipFile(output, 'r') as zip_ref:
        # Get the name of the first file in the archive
        original_name = zip_ref.namelist()[0]
        zip_ref.extractall(output_dir)

    # Remove the temporary zip file
    os.remove(output)

    # The path to the extracted file
    extracted_file = os.path.join(output_dir, original_name)

    print(f"File extracted as: {extracted_file}, saved to {output_dir}")
    return extracted_file


In [3]:
# create sub-directory under models/ and data/
checkpoint = download_and_unzip('1bJMOyA86CDayzwmU5KjlZnbhCXHUzO41', output_dir = os.getcwd() + "/models/trained_model.pt")
tokenizer = download_and_unzip('1UhsXL-ymGFy1fBftMvMbss2PGFzRxZV4', output_dir = os.getcwd() + "/data/trained_tokenizer.model")

Downloading...
From (original): https://drive.google.com/uc?id=1bJMOyA86CDayzwmU5KjlZnbhCXHUzO41
From (redirected): https://drive.google.com/uc?id=1bJMOyA86CDayzwmU5KjlZnbhCXHUzO41&confirm=t&uuid=1e6edaa9-2d4b-4b87-8d44-04b971c25072
To: /content/models/trained_model.pt/temp.zip
100%|██████████| 182M/182M [00:02<00:00, 66.9MB/s]


File extracted as: /content/models/trained_model.pt/trained_model_tok32000.pt, saved to /content/models/trained_model.pt


Downloading...
From: https://drive.google.com/uc?id=1UhsXL-ymGFy1fBftMvMbss2PGFzRxZV4
To: /content/data/trained_tokenizer.model/temp.zip
100%|██████████| 500k/500k [00:00<00:00, 71.8MB/s]

File extracted as: /content/data/trained_tokenizer.model/tok32000.model, saved to /content/data/trained_tokenizer.model


### (2) Load pretrained model and test
* Load pretrained model and pretrained tokenizer
* Adjust TF32 precision
* Config parameters for generation & decoding

In [4]:
from contextlib import nullcontext
import torch
from model import ModelArgs, Transformer
from tokenizer import Tokenizer
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Use device: {device}")

# load checkpoint
checkpoint_dict = torch.load(checkpoint, map_location=device)
gptconf = ModelArgs(**checkpoint_dict['model_args'])
model = Transformer(gptconf)
state_dict = checkpoint_dict['model']
unwanted_prefix = '_orig_mod.' #the unwanted prefix was sometimes added during compiling
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict, strict=False)
model.eval()
model.to(device)

# load tokenizer
enc = Tokenizer(tokenizer_model=tokenizer)

# adjust precision
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
torch.backends.cuda.matmul.allow_tf32 = True #enables the use of TF32 for matrix multiplication operations within PyTorch when using CUDA
torch.backends.cudnn.allow_tf32 = True #enables the use of TF32 precision within the cuDNN library
device_type = 'cuda' if 'cuda' in device else 'cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

Use device: cpu


<ipython-input-4-ec5a4e0f16ce>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint, map_location=device)


#words: 32000 - BOS ID: 1 - EOS ID: 2


* **Generate sample texts**

In [7]:
## parameter configurations
num_samples = 1 # number of samples to draw (how many paragraphs?)
max_new_tokens = 1024 # number of tokens generated in each sample
temperature = 1.2 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 300 # retain only the top_k most likely tokens



In [9]:
# Generate texts
start = "your model is so trash! Do you know?"
start_ids = enc.encode(start, bos=True, eos=False)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(enc.decode(y[0].tolist()))
            print('---------------')

your model is so trash! Do you know? Does he win for sure?” 
A lady came over to hear the news. "No, he may win," the lady said with a smile. 
Little did the lady think that saving her make a lot of money. She did her special job to protect the trash, caring for anyone it makes. Every day, she would and try to help with anything she dared. flowers bloom in the garden and made the neighborhood applauding. 
Many people also offered politely compliment the lady. They said things like she would never pick until the flowers were picked up. Luckily, they never had to refuse the man's decision.  Everyone worked together in a unique way – labeled this was exactly what they wanted and let it stay. 
The town was successful and organized once again! 
The lady thanked all her helpers and decided to make even more brilliant math. Happy — and busy love for her love. People always admired her magic of building the paper no more. Once upon a time, there was a little squirrel named Sam. Sam had a nut t

## A Glance at Decoding from A Trained Model

### (1). The `generate()` function takes auto-regression procedures

* `generate` is a method under the model class. It takes a conditioning sequence of indices `idx`, which is a LongTensor of shape `(batch_size, sentence_length)`, and completes the sequence `max_new_tokens` times, feeding the predictions back into the model each time. The function is often operated under `model.eval()` mode.
* **AR (auto-regression) generation** **by considering the next token sampled from a softmax layer**
* `@torch.inference_mode()` is an alternative to `with torch.no_grad()` to ***disable gradient calculation !***

In [ ]:
@torch.inference_mode()
def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
    for _ in range(max_new_tokens):
        idx_cond = idx if idx.size(1) <= self.params.max_seq_len else idx[:, -self.params.max_seq_len:]
        logits = self(idx_cond)
        logits = logits[:, -1, :] # crop to just the final time step
        if temperature == 0.0:
            _, idx_next = torch.topk(logits, k=1, dim=-1)
        else:
            logits = logits / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

* **To customize a new generation approach

In [ ]:
def custom_generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
    # Custom generation logic here
    for _ in range(max_new_tokens):
        ...
    return idx

# Assign the custom generate method to the model instance
model.generate = custom_generate.__get__(model, Transformer)

### (2) A closer look at `generate()` function component

1. **Greedy decoding** : Always pick the next word *with the highest probability*. This method can lead to repetitive and predictable text. `temperature = 0.0`

```
if temperature == 0.0:
    _, idx_next = torch.topk(logits, k=1, dim=-1)
    idx = torch.cat((idx, idx_next), dim=1)
```

2. **Temperature Scaling** :  A lower temperature makes the distribution peakier **(more greedy)**, while a higher temperature makes the distribution flatter **(more random)**.

```
logits = logits / temperature
probs = F.softmax(logits, dim=-1)
idx_next = torch.multinomial(probs, num_samples=1)
idx = torch.cat((idx, idx_next), dim=1)

```

3. **Top-k Sampling**: Limits the next word choices to the top k most probable words ($p_1 \ge p_2 \ge ... \ge p_k$):

```
if top_k is not None:
    values, indices = torch.topk(logits, k=top_k)
    logits[logits < values[:, [-1]]] = -float('Inf')
    probs = F.softmax(logits, dim=-1)
    idx_next = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, idx_next), dim=1)
```
4. **Top-p (Nucleus) Sampling**: This approach chooses the **smallest set** of words whose cumulative probability exceeds the probability $p$, a threshold.

```
cum_probs = torch.cumsum(F.softmax(logits, dim=-1), dim=-1)
threshold = torch.rand(1).item()
idx_next = torch.min((cum_probs > threshold).nonzero(as_tuple=True)[1])
idx = torch.cat((idx, idx_next.unsqueeze(0)), dim=1)
```

5. **Beam-search decoding** : Beam search *maintains multiple hypotheses* (the “beam”) at each step and expands them further by exploring several possible next steps. This strategy balances between *breadth (diversity) and depth (accuracy)*.

```
beam_width = 5
candidates = [idx]
for _ in range(max_new_tokens):
    all_candidates = []
    for candidate in candidates:
        logits = self(candidate)
        probs = F.softmax(logits[:, -1, :], dim=-1)
        top_probs, top_idx = torch.topk(probs, k=beam_width)
        for i in range(beam_width):
            next_candidate = torch.cat((candidate, top_idx[:, i:i+1]), dim=1)
            all_candidates.append((next_candidate, top_probs[:, i].item()))
    ordered = sorted(all_candidates, key=lambda x: x[1], reverse=True)
    candidates = [x[0] for x in ordered[:beam_width]]
idx = candidates[0]
```

## Tokenization and Vocabulary

### Vocabulary Construction WITH
* **Word2Vec** (static)
* **CBOW & Skip Gram**
* **Byte Pair Encoding** (BPE): BPE iteratively merges the most frequent pair of bytes or characters in a dataset into a single new token, continuing this process until a specified number of merges (vocabulary size) has been reached

* **
-- `SentencePiece` is a package that implements subword tokenization. It is language-independent and can be used to train LLMs **without requiring pre-tokenized text**.

```
import sentencepiece as spm
spm.SentencePieceTrainer.train(input=DATA_PATH, model_prefix=SAVE_PATH, model_type="bpe", vocab_size=VOCAB_SIZE, input_format="text")
```

In [12]:
import os
from sentencepiece import SentencePieceProcessor

class Tokenizer:
    def __init__(self, tokenizer_model):
        assert os.path.isfile(tokenizer_model), "Tokenizer model file not found."
        self.sp_model = SentencePieceProcessor(model_file=tokenizer_model)

        # Basic vocabulary properties
        self.n_words = self.sp_model.vocab_size()  # Total vocabulary size
        self.bos_id = self.sp_model.bos_id()       # Beginning of sentence token
        self.eos_id = self.sp_model.eos_id()       # End of sentence token
        self.pad_id = self.sp_model.pad_id()       # Padding token

    def encode(self, text, bos=True, eos=True):
        # Encodes text into a list of token ids
        tokens = self.sp_model.encode(text)
        if bos:
            tokens = [self.bos_id] + tokens
        if eos:
            tokens += [self.eos_id]
        return tokens

    def decode(self, tokens):
        # Decodes a list of token ids back into text
        return self.sp_model.decode(tokens)

# tok = Tokenizer(tokenizer_model)
# use tok.encode(text, bos=True, eos=False)
# use tok.decode(list_of_integers)

## Transformer Architecture

```
Transformer(
  (tok_embeddings): Embedding(32000, 288) #
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=288, out_features=288, bias=False)
        (wk): Linear(in_features=288, out_features=288, bias=False)
        (wv): Linear(in_features=288, out_features=288, bias=False)
        (wo): Linear(in_features=288, out_features=288, bias=False)
        (attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=288, out_features=768, bias=False)
        (w2): Linear(in_features=768, out_features=288, bias=False)
        (w3): Linear(in_features=288, out_features=768, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=288, out_features=32000, bias=False)
)
```

* `Embedding(32000, 288)`: Maps input tokens from a vocabulary of **32,000 to 288-dimensional embeddings**.

* `Dropout(p=0.0)`: Applies dropout with a probability of 0.0 (effectively no dropout in this setup).

* 6 x TransformerBlock: Six layers of Transformer blocks, each containing:

**Attention Mechanism**

* `Linear(in_features=288, out_features=288)`: Four linear transformations (for queries, keys, values, and output) in the multi-head attention mechanism, all with 288 features.

* `Dropout(p=0.0)`: Two dropout layers for attention and residual dropout, both set to 0.0 probability.

**FeedForward Network**

* `Linear(in_features=288, out_features=768)`: First linear layer of the feed-forward network.

* `Linear(in_features=768, out_features=288)`: Second linear layer that projects back to 288 features.

* `Dropout(p=0.0)`: Dropout layer in the feed-forward network.

**Normalization (LN)**

* `RMSNorm()`: Normalization layer for both the output of the attention block and the feed-forward network.

**Final Normalization**

* `RMSNorm()`: Normalization layer after the last Transformer block.

**Output Projection**

* `Linear(in_features=288, out_features=32000)`: Linear layer that projects the output of the Transformer to a vocabulary size of 32,000.